## Load packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import pandas_profiling
import datetime
%matplotlib inline
from io import StringIO
import os
import json
import re

### Load the file

In [7]:
# we can deal with it just in the memory
# df = pd.read_json(r'C:\Users\us61565\Desktop\Internal\Kaggle\sc_data_science_challenge.json')
os.chdir(r'C:\Users\us61565\Desktop\Internal\Kaggle')

# Reading the json as a dict
with open('sc_data_science_challenge.json') as json_data:
    data = json.load(json_data)

### Make a dataframe

In [18]:
# Had to make everything of an equal length first 
def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

In [19]:
data1 = pad_dict_list(data, 1) # using the function from above

In [180]:
df = pd.DataFrame.from_dict(data1).T

In [32]:
cols = df.T.iloc[0:15,1]  # saving columns' names
cols

In [38]:
cols_2_split = df.T.iloc[0:,0] # data to split

In [187]:
# make sure seconds are good. Remove from the final
original = cols_2_split.astype(str).str.split(', ', expand = True) 

In [189]:
original.columns = cols

In [190]:
original.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,[1467398683,'GB','204.0.20','tracks','content-based','b47911d0e80d1a8a959a2b726654bbfa','Dance & Electronic',1466640000,413933,413933,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
1,[1467398924,'GB','204.0.20','tracks','content-based','1f5e4bc82ba7fc802f780870f9b69191','HipHop & R&B',1466726400,402640,110722,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
2,[1467820636,'GB','204.0.20','users','content-based','092495283958e5d00f61cf2afe970d83','Pop',1467072000,494811,193818,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
3,[1467401951,'US','36.0.8','collection','content-based','211cd7be59941679f5ea847879b3b19b','HipHop & R&B',1402272000,361585,361585,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]
4,[1467402159,'US','36.0.8','collection','content-based','0e7ca1a7524aba84473540fa0d0ec75f','Pop',1448668800,373700,160162,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]


In [50]:
# remove
# t1 = cols_2_split.astype(str).map(lambda x: x[0]).str.rsplit(', ').str

In [62]:
df_values = cols_2_split.astype(str).str.split(', ', expand = True) 

In [68]:
# Giving names to columns
df_values.columns = cols

In [193]:
df_values.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,2016-07-01 18:44:43,GB,204.0.20,tracks,content-based,b47911d0e80d1a8a959a2b726654bbfa,Dance & Electronic,2016-06-23,413.933,413.933,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
1,2016-07-01 18:48:44,GB,204.0.20,tracks,content-based,1f5e4bc82ba7fc802f780870f9b69191,HipHop & R&B,2016-06-24,402.640,110.722,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
2,2016-07-06 15:57:16,GB,204.0.20,users,content-based,092495283958e5d00f61cf2afe970d83,Pop,2016-06-28,494.811,193.818,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
3,2016-07-01 19:39:11,US,36.0.8,collection,content-based,211cd7be59941679f5ea847879b3b19b,HipHop & R&B,2014-06-09,361.585,361.585,000264c5b34ad04999535c8b6a8b86e2,2013-09-17,HipHop & R&B,112239.18,10.5
4,2016-07-01 19:42:39,US,36.0.8,collection,content-based,0e7ca1a7524aba84473540fa0d0ec75f,Pop,2015-11-28,373.700,160.162,000264c5b34ad04999535c8b6a8b86e2,2013-09-17,HipHop & R&B,112239.18,10.5


In [103]:
# test = df_values

In [111]:
test.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,[1467398683,'GB','204.0.20','tracks','content-based','b47911d0e80d1a8a959a2b726654bbfa','Dance & Electronic',1466640000,413933,413933,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
1,[1467398924,'GB','204.0.20','tracks','content-based','1f5e4bc82ba7fc802f780870f9b69191','HipHop & R&B',1466726400,402640,110722,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
2,[1467820636,'GB','204.0.20','users','content-based','092495283958e5d00f61cf2afe970d83','Pop',1467072000,494811,193818,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
3,[1467401951,'US','36.0.8','collection','content-based','211cd7be59941679f5ea847879b3b19b','HipHop & R&B',1402272000,361585,361585,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]
4,[1467402159,'US','36.0.8','collection','content-based','0e7ca1a7524aba84473540fa0d0ec75f','Pop',1448668800,373700,160162,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]


In [192]:
df_values.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,2016-07-01 18:44:43,GB,204.0.20,tracks,content-based,b47911d0e80d1a8a959a2b726654bbfa,Dance & Electronic,2016-06-23,413.933,413.933,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
1,2016-07-01 18:48:44,GB,204.0.20,tracks,content-based,1f5e4bc82ba7fc802f780870f9b69191,HipHop & R&B,2016-06-24,402.640,110.722,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
2,2016-07-06 15:57:16,GB,204.0.20,users,content-based,092495283958e5d00f61cf2afe970d83,Pop,2016-06-28,494.811,193.818,0001d5755511cae40e5b32d83a3decce,2011-12-25,HipHop & R&B,617270.94,22.5
3,2016-07-01 19:39:11,US,36.0.8,collection,content-based,211cd7be59941679f5ea847879b3b19b,HipHop & R&B,2014-06-09,361.585,361.585,000264c5b34ad04999535c8b6a8b86e2,2013-09-17,HipHop & R&B,112239.18,10.5
4,2016-07-01 19:42:39,US,36.0.8,collection,content-based,0e7ca1a7524aba84473540fa0d0ec75f,Pop,2015-11-28,373.700,160.162,000264c5b34ad04999535c8b6a8b86e2,2013-09-17,HipHop & R&B,112239.18,10.5


### Cleaning the dataset

In [ ]:
# Square brackets
df_values['ts'] = df_values['ts'].str[1:] # removing the first bracket
df_values.iloc[:,-1] = df_values.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket

In [ ]:
# removing quotes
for a in df_values.iloc[:,1:7].columns.tolist():
    df_values[a] = df_values[a].astype(str).str.replace('\'', '')

for b in df_values[['listener_id','listener_top_genre_category_listened']].columns.tolist():
    df_values[b] = df_values[b].astype(str).str.replace('\'', '')    

In [ ]:
# unix timestamp to date
df_values['ts'] = pd.to_datetime(df_values['ts'], unit='s')
df_values['track_upload_date'] = pd.to_datetime(df_values['track_upload_date'], unit='s')
df_values['listener_signup_date'] = pd.to_datetime(df_values['listener_signup_date'], unit='s')

In [ ]:
# Milliseconds to seconds for now
df_values['track_duration'] = df_values['track_duration'].astype(float)/1000
df_values['listen_duration'] = df_values['listen_duration'].astype(float)/1000

In [ ]:
"""The above approach won't work with listener_prev_month_listening_time because it has a non-numeric value for customers who are 
first month users. Here is a work-around"""

df_values['listener_prev_month_listening_time'] = pd.to_numeric(df_values['listener_prev_month_listening_time'], errors='ignore')
df_values['listener_prev_month_listening_time'].fillna(0, inplace = True)
df_values['listener_prev_month_listening_time'] = df_values['listener_prev_month_listening_time'].astype(float)/1000

In [194]:
# let's check the NaN situation
df_values[df_values.isnull().any(axis=1)]

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened


In [195]:
df_values.dtypes

columns
ts                                               datetime64[ns]
country_code                                             object
client_version                                           object
listening_context                                        object
recommender_algorithm_name                               object
track_id                                                 object
track_genre_category                                     object
track_upload_date                                datetime64[ns]
track_duration                                          float64
listen_duration                                         float64
listener_id                                              object
listener_signup_date                             datetime64[ns]
listener_top_genre_category_listened                     object
listener_prev_month_listening_time                      float64
listener_prev_month_avg_daily_tracks_listened            object
dtype: object

In [197]:
# writing a .csv
df_values.to_csv(r'C:\Users\us61565\Desktop\Internal\Kaggle\df_values.csv', index=False)

In [ ]:
# cleaning columns from brackets, quotes, etc.
# df['result'] = df['result'].str.replace(r'\D', '')
# df_values['ts'] = df_values['ts'].str.replace(r'[','')
# df_values['ts'] = df_values['ts'].str[1:] # removing the first bracket
df_values.iloc[:,-1] = df_values.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket
# df_values['country_code'].apply(lambda s: s.str.replace('\'', ''))
# df_values['client_version'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))

In [ ]:
# test['country_code'].apply(lambda s: s.str.replace('\'', ''))
# test['country_code'].astype(str).str.replace('\'', '')

In [122]:
# removing brackets:
# test['ts'] = test['ts'].str[1:]
# test.iloc[:,-1] = test.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket


# # removing quotes
# for a in test.iloc[:,1:7].columns.tolist():
#     test[a] = test[a].astype(str).str.replace('\'', '')

# for b in test[['listener_id','listener_top_genre_category_listened']].columns.tolist():
#     test[b] = test[b].astype(str).str.replace('\'', '')    


In [ ]:
# unix timestamp to date
# test['ts'] = pd.to_datetime(test['ts'], unit='s')
# test['track_upload_date'] = pd.to_datetime(test['track_upload_date'], unit='s')
# test['listener_signup_date'] = pd.to_datetime(test['listener_signup_date'], unit='s')


In [177]:
# converting milliseconds to seconds for now
# test['track_duration'] = test['track_duration'].astype(float)/1000
# test['listen_duration'] = test['listen_duration'].astype(float)/1000
# test['listener_prev_month_listening_time'] = test['listener_prev_month_listening_time'].astype(float)/1000

In [137]:
# test['listener_prev_month_listening_time'].astype(str).applymap(lambda x: isinstance(x, (int, float)))
# test['listener_prev_month_listening_time'] = test['listener_prev_month_listening_time'].astype(str)

In [ ]:
# test['listener_prev_month_listening_time'].astype(str).apply(lambda x: isinstance(x, (str)))

In [141]:
# test['listener_prev_month_listening_time']

In [ ]:
# test.head()

In [174]:
# patterns= [r'\D+'] # non integers
# # for p in patterns:
# #     match=  re.findall(p, test['listener_prev_month_listening_time'])
# #     print(match)

# # test['listener_prev_month_listening_time'].str.contains(patterns, regex=True)
# # test['listener_prev_month_listening_time'] = df_values['listener_prev_month_listening_time']
# test['listener_prev_month_listening_time'] = pd.to_numeric(test['listener_prev_month_listening_time'], errors='ignore')
# test['listener_prev_month_listening_time'].fillna(0, inplace = True)
# # test['listener_prev_month_listening_time'].apply(lambda x: any(pd.Series(x).str.contains(patterns)))

In [ ]:
# test[test.isnull().any(axis=1)]